In [ ]:
import numpy as np # used for scientific computing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd # used for data analysis and manipulation
import matplotlib.pyplot as plt # used for visualization and plotting
import matplotlib.cm as cm
import scipy as sp
import pprint
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn import metrics
import dtreeviz.trees
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy import stats
import pprint



check for min coefficients
split to train/test by applicant, not randomally
validity of data (preprocessing)

In [ ]:
df = pd.read_csv('inner_data.csv')
column_headers = list(df.columns.values)
data = df.to_numpy()
len(column_headers)
# data

In [ ]:
data = data[:, 1:]
data = data[:, 1:]
# X = np.delete(data, feature, axis=1)
# y = data[:, feature]
column_headers.pop(0)
column_headers.pop(0)
data.shape
# len(column_headers)

In [ ]:
def provide_best_features(X_train_poly, X_test_poly, y_train, y_test, num_feat, thresh=0.1):
    # Train the polynomial regression model
    model = LinearRegression()
    # print(X_train_poly.shape, y_train.shape)
    model.fit(X_train_poly, y_train)
    # Predict on the testing set
    y_pred = model.predict(X_test_poly)

    # # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", "{:.4f}".format(mse))
    pred_comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    pred_comparison_df["True/False"] = np.abs(pred_comparison_df["Actual"] - pred_comparison_df["Predicted"]) < thresh        
    print("Accuracy: ","{:.2f}".format(100*pred_comparison_df["True/False"].value_counts(normalize=True)[0]),"%")
    
    feature_coefficients = np.abs(model.coef_)
    # Get the indices of the top 5 features with the highest coefficients
    top5_indices = np.argsort(feature_coefficients)[-num_feat:]
    # print(top5_indices)
    # Get the names of the top 5 features
    # top5_features = poly_names[top5_indices]  # Replace 'feature_names' with the actual names of your features

    # Print the top 5 features
    # print("Top 5 Features:")
    # for feature in top5_features:
    #     print(feature)
    return top5_indices.tolist(), mse, pred_comparison_df["True/False"].value_counts(normalize=True)[0]

In [ ]:
def linear_to_poly_data(X_train, X_test, degree):
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree)
    X_train_poly = poly_features.fit_transform(X_train)
    X_test_poly = poly_features.transform(X_test)
    poly_names = poly_features.get_feature_names_out()
    return X_train_poly, X_test_poly, poly_names
    

In [ ]:
def split_data_by_feature(df, feature):
    data =df.to_numpy()
    data = data[:, 1:]
    data = data[:, 1:]
    X = np.delete(data, feature, axis=1)
    y = data[:, feature]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def split_data_by_applicant(df, feature):
    applicants = df.Applicant.unique()
    applicants_train, applicants_test = train_test_split(applicants, test_size=0.2, random_state=42)
    data_train = df[df.Applicant.isin(applicants_train)].copy()
    data_test = df[df.Applicant.isin(applicants_test)].copy()
    data_train.drop(columns=['Question','Applicant'], inplace=True)
    data_test.drop(columns=['Question','Applicant'], inplace=True)

    data_train=data_train.to_numpy()
    data_test=data_test.to_numpy()

    X_train, X_test, y_train, y_test = np.delete(data_train, feature, axis=1),np.delete(data_test, feature, axis=1), data_train[:, feature],data_test[:, feature]
    return X_train, X_test, y_train, y_test

In [ ]:
def run_model_by_feature(df, degree, thresh, num_feat):
    best_features = []
    best_features_to_predict = {}
    for feature in range(df.shape[1]-2):
        print("FEATURE TO PREDICT: ", feature)
        X_train, X_test, y_train, y_test = split_data_by_feature(df, feature)
        X_train_poly, X_test_poly, poly_names = linear_to_poly_data(X_train, X_test, degree)
        model_eval = provide_best_features(X_train_poly, X_test_poly, y_train, y_test, num_feat)
        top_features = poly_names[model_eval[0]].tolist()
        best_features.append(model_eval[0])

        print("\npredict only with TOP 5 features:")
        X_train_only_best, X_test_only_best = X_train_poly[:,best_features[feature]], X_test_poly[:, best_features[feature]]
        limited_model_eval = provide_best_features(X_train_only_best, X_test_only_best, y_train, y_test, num_feat)
        if limited_model_eval[2]/model_eval[2] >= thresh:
            best_features_to_predict[feature] = {'feature names':top_features,'feature numbers':model_eval[0], 'mse':model_eval[1],'accuracy':model_eval[2],'limited model mse': limited_model_eval[1],'limited model accuracy': limited_model_eval[2], 'ratio':limited_model_eval[2]/model_eval[2]}
        print("\n\n")

    pprint.pprint(best_features_to_predict)
    return best_features_to_predict


In [ ]:
def run_model_by_applicant(df, degree, thresh, num_feat):
    best_features = []
    best_features_to_predict = {}
    for feature in range(df.shape[1]-2):
        print("FEATURE TO PREDICT: ", feature)
        X_train, X_test, y_train, y_test = split_data_by_applicant(df, feature)
        X_train_poly, X_test_poly, poly_names = linear_to_poly_data(X_train, X_test, degree)
        model_eval = provide_best_features(X_train_poly, X_test_poly, y_train, y_test, num_feat)
        top_features = poly_names[model_eval[0]].tolist()
        best_features.append(model_eval[0])

        print("\npredict only with TOP features:")
        X_train_only_best, X_test_only_best = X_train_poly[:,best_features[feature]], X_test_poly[:, best_features[feature]]
        limited_model_eval = provide_best_features(X_train_only_best, X_test_only_best, y_train, y_test, num_feat)
        if limited_model_eval[2]/model_eval[2] >= thresh:
            best_features_to_predict[feature] = {'feature names':top_features,'feature numbers':model_eval[0], 'mse':model_eval[1],'accuracy':model_eval[2],'limited model mse': limited_model_eval[1],'limited model accuracy': limited_model_eval[2], 'ratio':limited_model_eval[2]/model_eval[2]}
        print("\n\n")

    pprint.pprint(best_features_to_predict)
    return best_features_to_predict


In [ ]:
# run the program
degree = 1
thresh = 0.9
num_feat = 2
best_feat = run_model_by_feature(df,degree, thresh,num_feat)

In [ ]:
# run the program
degree = 1
thresh = 0.9
num_feat = 2
best_feat = run_model_by_applicant(df,degree, thresh,num_feat)

In [ ]:
len(best_feat)

In [ ]:
df = pd.read_csv('inner_data.csv')
column_headers = list(df.columns.values)
data1= df.groupby('Applicant').mean()
data1.drop(columns=['Question'], inplace=True)
# data1 = data1.to_numpy()
# data1.shape
data1 = data1.to_numpy()


In [ ]:
# run the program
degree = 1
thresh = 0.9
num_feat = 2
best_features = []
best_features_to_predict = {}

# for feature in range(X_train.shape[1]):
# def test_model_by_feature(data, )
for feature in range(82):
    print("FEATURE TO PREDICT: ", feature)
    X_train, X_test, y_train, y_test = split_data_by_feature(df, feature)
    X_train_poly, X_test_poly, poly_names = linear_to_poly_data(X_train, X_test, degree)
    model_eval = provide_best_features(X_train_poly, X_test_poly, y_train, y_test, num_feat)
    top_features = poly_names[model_eval[0]].tolist()
    best_features.append(model_eval[0])

    print("\npredict only with TOP 5 features:")
    X_train_only_best, X_test_only_best = X_train_poly[:,best_features[feature]], X_test_poly[:, best_features[feature]]
    limited_model_eval = provide_best_features(X_train_only_best, X_test_only_best, y_train, y_test, num_feat)
    if limited_model_eval[2]/model_eval[2] >= .9:
        best_features_to_predict[feature] = {'feature names':top_features,'feature numbers':model_eval[0], 'mse':model_eval[1],'accuracy':model_eval[2],'limited model mse': limited_model_eval[1],'limited model accuracy': limited_model_eval[2], 'ratio':limited_model_eval[2]/model_eval[2]}
    print("\n\n")

pprint.pprint(best_features_to_predict)


In [ ]:
len(best_features_to_predict)

In [ ]:
df = pd.read_csv('inner_data.csv')
column_headers = list(df.columns.values)
def split_data_by_applicant(df, feature):
    applicants = df.Applicant.unique()
    applicants_train, applicants_test = train_test_split(applicants, test_size=0.2, random_state=42)
    # print(applicants_train)
    data_train = df[df.Applicant.isin(applicants_train)].copy()
    data_test = df[df.Applicant.isin(applicants_test)].copy()
    data_train.drop(columns=['Question','Applicant'], inplace=True)
    data_test.drop(columns=['Question','Applicant'], inplace=True)
    # print(data_train, data_test)

    data_train=data_train.to_numpy()
    data_test=data_test.to_numpy()

    X_train, X_test, y_train, y_test = np.delete(data_train, feature, axis=1),np.delete(data_test, feature, axis=1), data_train[:, feature],data_test[:, feature]
    return X_train, X_test, y_train, y_test
    


In [ ]:
# # run the program
# degree = 1
# thresh = 0.9
# num_feat = 2
# best_features = []
# best_features_to_predict = {}
# df = pd.read_csv('inner_data.csv')

# # for feature in range(X_train.shape[1]):
# for feature in range(82):
#     print("FEATURE TO PREDICT: ", feature)
#     X_train, X_test, y_train, y_test = split_data_by_applicant(df, feature)
#     X_train_poly, X_test_poly, poly_names = linear_to_poly_data(X_train, X_test, degree)
#     model_eval = provide_best_features(X_train_poly, X_test_poly, y_train, y_test, num_feat)
#     top_features = poly_names[model_eval[0]].tolist()
#     best_features.append(model_eval[0])

#     print("\npredict only with TOP 5 features:")
#     X_train_only_best, X_test_only_best = X_train_poly[:,best_features[feature]], X_test_poly[:, best_features[feature]]
#     limited_model_eval = provide_best_features(X_train_only_best, X_test_only_best, y_train, y_test, num_feat)
#     if limited_model_eval[2]/model_eval[2] >= thresh:
#         best_features_to_predict[feature] = {'feature names':top_features,'feature numbers':model_eval[0], 'mse':model_eval[1],'accuracy':model_eval[2],'limited model mse': limited_model_eval[1],'limited model accuracy': limited_model_eval[2], 'ratio':limited_model_eval[2]/model_eval[2]}
#     print("\n\n")

# pprint.pprint(best_features_to_predict)


In [ ]:
len(best_features_to_predict)